In [2]:
from collections import defaultdict
from IPython.display import display
from ipywidgets import interact, interact_manual
import ipywidgets as widgets
import pyfiglet

from refactorings_detection import *
from util import *
from study_notebook_helper import *


def get_containing_class_of(repo: LocalRepo, path: str) -> str:
    return repo.get_tree().find_node(path).get_containing_class_node().get_path()


ignored_types = SerializedWrap(set(), "../refactorings/ignored_refactoring_types.pickle")


@interact_manual.options(manual_name="Go!")(info=[
    "jfree/jfreechart:v1.5.3::v1.5.0",
    "junit-team/junit4:r4.13.2::r4.12",
    "junit-team/junit4:r4.13.2::r4.10",
    "junit-team/junit4:r4.13.2::r4.6",
])
def main(info):
    repo_name, old_version = info.split("::")
    # 1 get list of refactorings for each old class
    #   a refactoring contains a type and a set of locations within this and other old classes
    # 2 cached, interactively ask the user whether this class has been "refactored to fix its modular structure / architecture"
    #   showing old and new code of the class, and the list of refactorings (grouped by other old location), links to GitHub old and new version of the other locations
    #   While doing so, allow users to filter more refactoring types (also persisted)
    repo = LocalRepo(repo_name)
    repo.get_tree(old_version)
    repo.get_tree()
    old = repo.get_commit(old_version).hexsha
    new = repo.get_head_commit().hexsha

    RefactoringType = Tuple[str, str] # type_name, desc
    results: Dict[str, Dict[str, List[RefactoringType]]] = defaultdict(lambda: defaultdict(lambda: []))  # old class, old other class, ref
    for commit in get_raw_refactorings_per_commit(repo, old, new):
        for ref in commit["refactorings"]:
            type_name, description = ref["type"], ref["description"]
            description = remove_prefix(description, type_name).lstrip()
            
            if type_name not in ignored_types.data:
                for left_side_path in set(refactoring_process_path(repo, ref["leftSideLocations"])):
                    left_side_class_node = repo.get_tree().find_node(left_side_path).get_containing_class_node()
                    left_side_class_java_name = left_side_class_node.get_java_name()
                    shorter_description = remove_suffix(description, " in class " + left_side_class_java_name)
                    shorter_description = remove_suffix(shorter_description, " from class " + left_side_class_java_name)
                    for right_side_path in set(refactoring_process_path(repo, ref["rightSideLocations"])):
                        results[left_side_class_node.get_path()][right_side_path].append((type_name, shorter_description))
    print(len(results))
    all_types = set(type_name for class_info in results.values() for refs in class_info.values() for type_name, desc in refs)
    
    @interact_manual.options(manual_name="Ignore type!")(ignore=[None] + sorted(all_types))
    def ignore_type(ignore):
        if ignore is None:
            return
        print(f"Ignoring {ignore}!")
        ignored_types.data.add(ignore)
        ignored_types.save()
        print(pyfiglet.figlet_format("Reloading..."))
        main.widget.manual_button.click()
    
    @interact_manual.options(manual_name="Unignore type!")(unignore=[None] + sorted(ignored_types.data))
    def ignore_type(unignore):
        if unignore is None:
            return
        print(f"Ignoring {unignore}!")
        ignored_types.data.remove(unignore)
        ignored_types.save()
        print(pyfiglet.figlet_format("Reloading..."))
        main.widget.manual_button.click()
        
        
    confirmed_refactorings = SerializedWrap(dict(), f"""../refactorings/confirmed_{info.replace("/", "_")}_.pickle""") # Dict from class_path to bool
    
    class_path_dropdown = widgets.Dropdown(description='class_path', options=[])
    
    def update_options():
        class_path_dropdown.options = [cp for cp in results.keys() if cp not in confirmed_refactorings.data]
    
    update_options()
    HEADING_TEXT = "Has this class been refactored to fix its modular structure / architecture?"
    SCROLL_TO_TOP = f"""<script>[...document.getElementsByTagName("h1")].find(e => e.innerText === "{HEADING_TEXT}").scrollIntoView();</script>"""
    @interact(class_path=class_path_dropdown)
    def show_refactoring(class_path: str):
        confirmed_count = sum(confirmed_refactorings.data.values())
        denied_count = len(confirmed_refactorings.data) - confirmed_count
        print(f"""Status: {len(results)} possibly refactored classes. Of those, {confirmed_count} confirmed, {denied_count} denied, and {len(class_path_dropdown.options)} not yet categorized.""")
        if class_path is None:
            print("All done!")
            print(sorted(list(results.keys())))
            print(sorted(list(confirmed_refactorings.data.keys())))
            return

        print("\n" * 5)
        print_html(f"<h1>{HEADING_TEXT}</h1>")
        print_html(SCROLL_TO_TOP)
        print("\n" * 5)
        
        print_html(f"""{OWN_STYLE}<br>
            <center>
                Location of class within the project tree:<br>
                <br>
                {make_path_html(repo, "", class_path.split("/"))}
            </center>
            <br><br>
        """)

        print_html("<h4>Refactorings:</h4>")
        for other, refs in results[class_path].items():
            print(change_text_style(other, "bold+italic"))
            for type_name, desc in refs:
                print(f""" - {change_text_style(type_name, "bold")}: {desc}""")
        
        try:
            file_path = class_path.split(".java")[0] + ".java"        
            old_tree = repo.get_tree(old_version)
            new_tree = repo.get_tree()
            old_node = old_tree.find_node(class_path)
            new_node = new_tree.find_node(class_path)
            old_code = old_node.get_comment_and_own_text_formatted(repo.get_file(file_path, old_version))
            new_code = new_node.get_comment_and_own_text_formatted(repo.get_file(file_path))
            
            left_vis, right_vis = make_tree_comparison_html(old_node, new_node, lambda path, is_left: repo.url_for(path, old_version if is_left else None))
            
            link_html = f"""
                    <tr>
                      <td style="font-family: monospace"><center>Old: {path_html(repo, class_path, old_version)}</center></td>
                      <td style="font-family: monospace"><center>New: {path_html(repo, class_path)}</center></td>
                    </tr>"""
            print_html(f"""{PRISM_HIGHLIGHTING}
                <div style="max-height: 800px; overflow-y: scroll; border: 2px dashed #ccc; padding: 2px">
                  <table border="1" style="width: calc(0.95 * 100vw - 150px)">
                    {link_html}
                    <tr>
                      <td style="font-family: monospace"><center><div style="display: inline-block; text-align: left;">{left_vis}</div></center></td>
                      <td style="font-family: monospace"><center><div style="display: inline-block; text-align: left;">{right_vis}</div></center></td>
                    </tr>
                    {link_html}
                    <tr>
                      <td style="max-width: 50%"><pre><code class="language-java match-braces rainbow-braces">{format_code(old_code)}</code></pre></td>
                      <td style="max-width: 50%"><pre><code class="language-java match-braces rainbow-braces">{format_code(new_code)}</code></pre></td>
                    </tr>
                  </table>
                </div>""")
        except Exception as e:
            print(f"Cannot show old and new code for {class_path}!")
            print("Error:")
            print(e)
            print_html(f"""{path_html(repo, class_path, old_version)} :: {path_html(repo, class_path)}""")
                
        print("\n\n" + HEADING_TEXT)
        button_yes = widgets.Button(
            description='Yes!',
            button_style='success',
            icon='check-circle'
        )
        button_no = widgets.Button(
            description='No!',
            button_style='danger', # 'success', 'info', 'warning', 'danger', or ''
            icon='times-circle'
        )
        
        def submit(btn):
            confirmed_refactorings.data[class_path] = btn == button_yes
            confirmed_refactorings.save()
            update_options()

        button_yes.on_click(submit)
        button_no.on_click(submit)
        display(button_yes)
        display(button_no)
        
    # show_refactoring(list(results.keys())[0])


interactive(children=(Dropdown(description='info', options=('jfree/jfreechart:v1.5.3::v1.5.0', 'junit-team/jun…